# Retrieval and Reranking Logic:

## Retrieval Logic - Iteration 1:

- Lets say we have embeddings of articles stored in a vector db
- There is a given user query, we create an embedding of the query
- We extract certain keywords (mainly NOUNS) from user queries using basic NER (Stanza)
- We perform initial keyword based filtering on chunks using the chunk's metadata. eg.
    - Filtering on the Author's names
    - Filtering on the Journal's name
    - Filtering on the Year of Publication (if applicable)
- We then calculate the cosine similarity between the query embedding and the chunks embedding
- We fetch Top k (=5) ARTICLES with there top N (=5) matching chunks. 
- Then we perform the RERANKING to reduce the False Positives.

## Reranking Logic - Iteration 1:

Lets say, we have Top 5 Articles with top 5 similar chunks

Logic:
- Calculate simillarity score between those top k (=5) articles (full articles).
- This would give us k*(k-1)/2 (for k=5, =10) similarity scores.
- Classify those Article pairs in 1 of 4 clusters in a k X k Matrix (here, 5X5):
    - High Similarity
    - Moderate Similarity
    - Low Similarity
    - Unrelated  
- Example of a k X k matrix:
5 X 5 Matrix of Simillarity
    a1  a2  a3  a4  a5
a1  HS  LS  HS  LS  LS
a2  LS  HS  MS  HS  HS
a3  HS  MS  HS  LS  LS
a4  LS  HS  LS  HS  HS
a5  LS  HS  LS  HS  HS

Hence, 
a1 and a3 are HS
a2, a4, a5 are HS
(a2, a4, a5) are LS to a1
(a2, a4, a5) are LS to a3

Question:
- Which one out of (a1,a3) or (a2,a4,a5) is the actual and which one is Decoy?
- How to assign weightage to Similarity Score with User Query VS the Similarity Score with other articles

Possible Solution: Weighted Score
eg. Let's assume:
Similarity Score with User Query - 0.87
Similarity Score with Other Articles - 0.69 0.23 0.45 0.56 0.78
Avg Simillarity Score with other articles - 0.542
Number of Citations - 10 -> Norminalise it to a score between 0 and 1 - 0.2
Other Factors - 0.4 (If any??)

Weights:
SS with User Query - 0.7
SS with Other articles - 0.2
Other Factors - 0.1

Overall Simillarity Score = 0.87 * 0.7 + 0.67 * 0.2 + (0.2 + 0.4) * 0.1 = 0.803

- Finally, we can rank the Articles based on the Overall Simillarity Score

